<center>

<h1 style="text-align:center"> Lambda Calculus : Encodings </h1>
<h2 style="text-align:center"> CS3100 Fall 2019 </h2>
</center>

## Review

### Previously

* Semantics of untyped lambda calculus
  + β-reductions, reduction strategies, normal forms, extensionality
  
### Today

* Encodings
  + Booleans, Arithmetic, Pairs, Recursion

## Power of Lambdas

* Despite its simplicity, the lambda calculus is quite expressive: it is **Turing complete**!
* Means we can encode any computation we want
  + if we are sufficiently clever...
* Examples
  + Booleans & predicate logic.
  + Pairs
  + Lists
  + Natural numbers & arithmetic.
  
$\newcommand{\br}{\rightarrow_{\beta}}$

In [4]:
#use "init.ml"

let p = Lambda_parse.parse_string                                                   
let var x = Var x                                                                 
let app l =
  match l with 
  | [] -> failwith "ill typed app"
  | [x] -> x
  | x::y::xs -> List.fold_left (fun expr v -> App (expr, v)) (App(x,y)) xs
let lam x e = Lam(x,e)                                                              
                                                                                    
let eval ?(log=true) ?(depth=1000) s = 
     s  
  |> Eval.eval ~log ~depth Eval.reduce_normal 
  |> Syntax.string_of_expr

val p : string -> Syntax.expr = <fun>


val var : string -> Syntax.expr = <fun>


val app : Syntax.expr list -> Syntax.expr = <fun>


val lam : string -> Syntax.expr -> Syntax.expr = <fun>


val eval : ?log:bool -> ?depth:int -> Syntax.expr -> string = <fun>


In [5]:
p "\\x.x";;
var "x";;
app [var "x"; var "y"; var "z"];;
lam "x" (var "y");;

- : Syntax.expr = Lam ("x", Var "x")


- : Syntax.expr = Var "x"


- : Syntax.expr = App (App (Var "x", Var "y"), Var "z")


- : Syntax.expr = Lam ("x", Var "y")


## Booleans

In [6]:
let tru = p "\\t.\\f.t"
let fls = p "\\t.\\f.f"

val tru : Syntax.expr = Lam ("t", Lam ("f", Var "t"))


val fls : Syntax.expr = Lam ("t", Lam ("f", Var "f"))


* Now we can define a `test` function such that
  + `test tru v w` $\br$ `v`
  + `test fls v w` $\br$ `w`

In [7]:
let test = p "\\l.\\m.\\n.l m n"

val test : Syntax.expr =
  Lam ("l", Lam ("m", Lam ("n", App (App (Var "l", Var "m"), Var "n"))))


## Booleans

Now 

```ocaml
test tru v w
```

evaluates to

In [8]:
eval @@ app [test; tru; lam "x" (var "x"); lam "x" (lam "y" (var "x"))]

= (λl.λm.λn.l m n) (λt.λf.t) (λx.x) (λx.λy.x)
= (λm.λn.(λt.λf.t) m n) (λx.x) (λx.λy.x)
= (λn.(λt.λf.t) (λx.x) n) (λx.λy.x)
= (λt.λf.t) (λx.x) (λx.λy.x)
= (λf.λx.x) (λx.λy.x)
= λx.x


- : string = "λx.x"


## Booleans

Similarly,

```ocaml
test fls v w
```

evaluates to

In [9]:
eval @@ app [test; fls; var "v"; var "w"]

= (λl.λm.λn.l m n) (λt.λf.f) v w
= (λm.λn.(λt.λf.f) m n) v w
= (λn.(λt.λf.f) v n) w
= (λt.λf.f) v w
= (λf.f) w
= w


- : string = "w"


## Booleans

`fls` itself is a function. `test fls v w` is equivalent to `fls v w`.

In [10]:
eval @@ app [fls; var "v"; var "w"]

= (λt.λf.f) v w
= (λf.f) w
= w


- : string = "w"


## Logical operators

```ocaml
and = λb.λc.b c fls
or = λb.λc.b tru c
not = λb.b fls tru
```

In [11]:
let and_ = lam "b" (lam "c" (app [var "b"; var "c"; fls]))
let or_ = lam "b" (lam "c" (app [var "b"; tru; var "c"]))
let not_ = lam "b" (app [var "b"; fls; tru])

val and_ : Syntax.expr =
  Lam ("b",
   Lam ("c", App (App (Var "b", Var "c"), Lam ("t", Lam ("f", Var "f")))))


val or_ : Syntax.expr =
  Lam ("b",
   Lam ("c", App (App (Var "b", Lam ("t", Lam ("f", Var "t"))), Var "c")))


val not_ : Syntax.expr =
  Lam ("b",
   App (App (Var "b", Lam ("t", Lam ("f", Var "f"))),
    Lam ("t", Lam ("f", Var "t"))))


## Logical Operators

In [12]:
eval @@ app [and_; tru; fls]

= (λb.λc.b c (λt.λf.f)) (λt.λf.t) (λt.λf.f)
= (λc.(λt.λf.t) c (λt.λf.f)) (λt.λf.f)
= (λt.λf.t) (λt.λf.f) (λt.λf.f)
= (λf.λt.λf.f) (λt.λf.f)
= λt.λf.f


- : string = "λt.λf.f"


The above is a **proof** for `true /\ false = false`

## Logical operators

Encode implies using standard formulation.

\\[
\begin{array}{rl}
 & p \implies q \equiv \neg p \vee q \\
\mathbf{Theorem 1.}  & \forall a,b.~ a \wedge b \implies a
\end{array}
\\]

In [13]:
let implies = lam "p" (lam "q" (app [or_; app [not_; var "p"]; var "q"]))
let thm1 = lam "a" (lam "b" (app [implies; app [and_; var "a"; var "b"]; var "a"]))

val implies : Syntax.expr =
  Lam ("p",
   Lam ("q",
    App
     (App
       (Lam ("b",
         Lam ("c",
          App (App (Var "b", Lam ("t", Lam ("f", Var "t"))), Var "c"))),
       App
        (Lam ("b",
          App (App (Var "b", Lam ("t", Lam ("f", Var "f"))),
           Lam ("t", Lam ("f", Var "t")))),
        Var "p")),
     Var "q")))


val thm1 : Syntax.expr =
  Lam ("a",
   Lam ("b",
    App
     (App
       (Lam ("p",
         Lam ("q",
          App
           (App
             (Lam ("b",
               Lam ("c",
                App (App (Var "b", Lam ("t", Lam ("f", Var "t"))), Var "c"))),
             App
              (Lam ("b",
                App (App (Var "b", Lam ("t", Lam ("f", Var "f"))),
                 Lam ("t", Lam ("f", Var "t")))),
              Var "p")),
           Var "q"))),
       App
        (App
          (Lam ("b",
            Lam ("c",
             App (App (Var "b", Var "c"), Lam ("t", Lam ("f", Var "f"))))),
          Var "a"),
        Var "b")),
     Var "a")))


## Logical operators

Prove $~~\forall a,b. a \wedge b \implies a~~$ by case analysis:

In [14]:
eval ~log:false (app [thm1; tru; tru])

- : string = "λt.λf.t"


In [15]:
eval ~log:false (app [thm1; tru; fls])

- : string = "λt.λf.t"


In [16]:
eval ~log:false (app [thm1; fls; tru])

- : string = "λt.λf.t"


In [17]:
eval ~log:false (app [thm1; fls; fls])

- : string = "λt.λf.t"


**QED.**

## Quiz

What is the lambda calculus encoding for `xor x y`

| x | y | xor x y |
|---|---|:-------:|
| T | T |    F    |
| T | F |    T    |
| F | T |    T    |
| F | F |    F    |

1. x x y
2. x (y tru fls) y
3. x (y fls tru) y
4. y x y

## Quiz

What is the lambda calculus encoding for `xor x y`

| x | y | xor x y |
|---|---|:-------:|
| T | T |    F    |
| T | F |    T    |
| F | T |    T    |
| F | F |    F    |

1. x x y
2. x (y tru fls) y
3. x (y fls tru) y ✅
4. y x y

## Pairs

In [ ]:
let mk_pair x y = (x,y)
let fst (x,y) = x
let snd (x,y) = y

## Pairs

* Encoding of a pair `(f,s)`
  + Pair Constructor : (f,s) = λf.λs.λb.b f s

In [ ]:
let pair = p "λf.λs.λb.b f s"

## Pairs

In [ ]:
eval @@ app [pair; var "v"; var "w"]

* The pair **value** is a function that takes a **boolean** as an argument and applies the elements of the pair to it.
* `b` is a boolean is a **convention** that we should follow.
  + No type safety.

## Pair accessor functions

* Recall that a pair value is a function `λb.b v w` 
  + where `v` and `w` are the first and second elements of the pair.
* We can define accessors `fst` and `snd` as follows:
  + fst = λp.p tru
  + snd = λp.p fls

In [ ]:
let fst = lam "p" (app [var "p"; tru])
let snd = lam "p" (app [var "p"; fls])

## Pair accessor functions

In [ ]:
eval ~log:true @@ app [fst; app[pair; var "v"; var "w"]]

In [ ]:
eval ~log:false @@ app [snd; app [pair; var "v"; var "w"]]

## Pair swap function

In OCaml,

```ocaml
let swap (x,y) = (y,x)
```

In lambda calculus,

```ocaml
swap = λp.λb.b (snd p) (fst p)
```

In [ ]:
let swap = lam "p" (lam "b" (app [var "b"; app [snd;var "p"]; app [fst; var "p"]]))

## Pair swap function

Let's try

```ocaml
fst (swap (v,w))
```

In [ ]:
eval ~log:false @@ app [fst; app [swap; app[pair; var "v"; var "w"]]]

## Natural numbers

* 0 = λs.λz.z
* 1 = λs.λz.s z
* 2 = λs.λz.s (s z)
* 3 = λs.λz.s (s (s z))

i.e., n = λs.λz.(apply `s` n times to `z`)

Also known as **Church numerals**.

## Natural numbers

In [ ]:
let zero = p ("λs.λz.z")
let one = p ("λs.λz.s z")
let two = p ("λs.λz.s (s z)")
let three = p ("λs.λz.s (s (s z))")

## Quiz

What will be the OCaml type of church encoded number 2: $\lambda s.\lambda z.s~(s ~z)$?

1. `('a -> 'b) -> 'a -> 'b`
2. `('a -> 'a) -> 'a -> 'a` 
3. `('a -> 'a) -> 'b -> int`
4. `(int -> int) -> int -> int`

## Quiz

What will be the OCaml type of church encoded number 2: $\lambda s.\lambda z.s~(s ~z)$?

1. `('a -> 'b) -> 'a -> 'b`
2. `('a -> 'a) -> 'a -> 'a` ✅
3. `('a -> 'a) -> 'b -> int`
4. `(int -> int) -> int -> int`

## Operations on numbers: Successor

Successor function is:

```ocaml
scc = λn.λs.λz.s (n s z)
```

In [ ]:
let scc = p ("λn.λs.λz.s (n s z)")

In [ ]:
eval @@ app [scc; zero]

## Operations on numbers : is_zero

Check if the given number is zero:

```ocaml
is_zero = λn.n (λy.fls) tru
```

In [ ]:
let is_zero = lam "n" (app [var "n"; lam "y" fls; tru])

## Operations on numbers : is_zero

In [ ]:
eval @@ app [is_zero; zero]

In [ ]:
eval @@ app [is_zero; one]

## Arithmetic

```ocaml
plus = λm.λn.λs.λz.m s (n s z)
mult = λm.λn.λs.λz.m (n s) z
```

In [ ]:
let plus = p ("λm.λn.λs.λz.m s (n s z)")
let mult = p ("λm.λn.λs.λz.m (n s) z")

## Arithmetic: addition

In [ ]:
eval @@ app [plus; one; two]

Proves 1 + 2 = 3. Can build a theory of arithmetic over lambda calculus.

## Arithmetic: multiplication

In [ ]:
eval @@ app [mult; two; three]

## Arithmetic: predecessor

It turns out predecessor function is much more tricky compared to successor. 

```ocaml
zz = pair zero zero
ss = λp. pair (snd p) (plus one (snd p))
```

```ocaml
zz = (0,0)
ss zz = (0,1)
ss (ss zz) = (1,2)
ss (ss (ss zz)) = (2,3)
```
etc.

## Arithmetic: predecessor

It turns out predecessor function is much more tricky compared to successor. 

```ocaml
zz = pair zero zero
ss = λp. pair (snd p) (plus one (snd p))
prd = λm. fst (m ss zz)
```

In [ ]:
let zz = app [pair; zero; zero]
let ss = lam "p" (app [pair; app [snd; var "p"]; app [plus; one; app [snd; var "p"]]])
let prd = lam "m" (app [fst; app [var "m"; ss; zz]])

## Arithmetic: Predecessor

In [ ]:
eval ~log:false @@ app [prd; three]

In [ ]:
eval ~log:false @@ app [prd; zero]

## Arithmetic: Subtraction

`sub` computes `m-n`:

```ocaml
sub = λm.λn.n prd m
```

Intuition: apply predecessor `n` times on `m`.

In [ ]:
let sub = lam "m" (lam "n" (app [var "n"; prd; var "m"]))

## Arithmetic: Subtraction

In [ ]:
eval ~log:false @@ app [sub; three; two]

In [ ]:
eval ~log:false @@ app [sub; two; three]

## Arithmetic: equal

* `m - n = 0` $\implies$ `m = n`.
  + But we operate on natural numbers.
  + `3 - 4 = 0` $\implies$ `3 = 4`.
* `m - n = 0 && n - m = 0` $\implies$ `m = n`.

In [ ]:
let equal = 
  let mnz = app [is_zero; app [sub; var "m"; var "n"]] in
  let nmz = app [is_zero; app [sub; var "n"; var "m"]] in
  lam "m" (lam "n" (app [and_; mnz; nmz]))

## Arithmetic: equal

In [ ]:
eval ~log:false @@ app [equal; two; two]

In [ ]:
eval ~log:false @@ app [equal; app[sub; three; two]; two]

In [ ]:
eval ~log:false @@ app [equal; app[sub; two; three]; zero]

## Fixed points

* Given a function $f$ and some value $x$, if $f(x) = x$ then $x$ is said to be a fixed point for $f$.
  + $f(x) = x^2$ has two fixed points 0 and 1. 
  + $f(x) = x + 1$ has no fixed points.
* For lambda calculus, $N$ is said to be a fixed point of $F$ if $F ~N =_{\beta} N$
  + In the untyped lambda calculus, every term F has a fixed point!

## Fixed points

* Let `D = λx.x x`, then
  + `D D = (λx.x x) (λx.x x)` $\rightarrow_{\beta}$ `(λx.x x) (λx.x x) = D D`.
* So `D D` is an infinite loop
  + In general, self-application is how you get looping

## Fixed points

$
\require{color}
\newcommand{\yb}[1]{\colorbox{yellow}{$#1$}}
$

Let $Y = \lambda f.(\lambda x.f ~(x ~x)) ~(\lambda x.f ~(x ~x))$, then

\\[
\begin{array}{rl}
& Y ~F = (\lambda \yb{f}.(\lambda x.f ~(x ~x)) ~(\lambda x.f ~(x ~x))) ~F \\
\rightarrow_{\beta} & (\lambda \yb{x}.F ~(x ~x)) ~\yb{(\lambda x.F ~(x ~x))} \\
\rightarrow_{\beta} & F \yb{((λx.F ~(x ~x)) ~(λx.F ~(x ~x)))} \\
\rightarrow_{\beta} & F ~(Y ~F)
\end{array}
\\]

* Therefore, `Y F = F(Y F)`. 
  + `Y F` is said to be the fixed point of `F`.
  + `Y F = F(Y F) = F(F(Y F)) = ...`
  + `Y` (`y`-combinator) can be used to achieve recursion.

## Fixed point: Factorial

```ocaml
fact = λf.λn.if n = 0 then 1 else n * f (n-1)
```

* Second argument `n` is the integer.
* First argument `f` is the function to call for the recursive case.
* We will use y-combinator to achieve recursion. 

## Fixed point: Factorial

\\[
\begin{array}{rl}
& (Y ~\text{fact}) ~1 \\ 
\rightarrow_{beta} & ~\text{fact} ~(Y ~\text{fact}) 1 \\
\rightarrow_{beta} & \text{if } 1 = 0 \text{ then } 1 \text{ else } 1 * ((Y \text{ fact}) ~0) \\
\rightarrow_{beta} & 1 * ((Y \text{ fact}) ~0) \\
\rightarrow_{beta} & 1 * (\text{fact } (Y \text{ fact}) ~0) \\
\rightarrow_{beta} & 1 * \text{if } 0 = 0 \text{ then } 1 \text{ else } 1 * ((Y \text{ fact}) ~0) \\
\rightarrow_{beta} & 1 * 1 \\
\rightarrow_{beta} & 1 \\
\end{array}
\\]

## Fixed point: Factorial


In [ ]:
let y = p "λf.(λx.f (x x)) (λx.f (x x))"
let fact = 
  let tst = app [is_zero; var "n"] in
  let fb = app [mult; var "n"; app [var "f"; app [prd; var "n"]]] in
  lam "f" (lam "n" (app [tst; one; fb]))

In [ ]:
eval ~log:false @@ app [y; fact; two]

## Quiz

The y-combinator Y = λf.(λx.f (x x)) (λx.f (x x)) is a fixed pointer combinator under which reduction strategy?

1. Call-by-value
2. Call-by-name
3. Both
4. Neither

## Quiz

The y-combinator Y = λf.(λx.f (x x)) (λx.f (x x)) is a fixed pointer combinator under which reduction strategy?

1. Call-by-value
2. Call-by-name ✅
3. Both
4. Neither

Under call-by-value, we will keep indefinitely expanding `Y F = F (Y F) = F (F (Y F)) = ...`.

## Fixed point: Z combinator

There is indeed a fixed point combinator for call-by-value called the Z combinator

```ocaml
Z = λf. (λx. f (λy. x x y)) (λx. f (λy. x x y))
```

which is just an $\eta$-expansion of the Y combinator

```ocaml
Y = λf. (λx. f (x x)) (λx. f (x x))
```

## Fixed point: Z combinator

\\[
\begin{array}{rl}
 & Z ~F = (λ\yb{f}.(λx.f ~(λy.x ~x ~y)) ~(λx.f ~(λy.x ~x ~y))) ~\yb{F} \\
\rightarrow_{\beta V} & (λ\yb{x}.F ~(λy.x ~x ~y)) ~\yb{(λx.F ~(λy.x ~x ~y))} \\
\rightarrow_{\beta V} & F ~(λy.\yb{(λx.F ~(λy.x ~x ~y)) ~(λx.F ~(λy.x ~x ~y))} ~y)  \\
\rightarrow_{\beta V} & F ~(λy. (Z ~F) ~y)
\end{array}
\\]

The $\eta$-expansion has prevented further reduction.

## Recursive data structures: Lists

* We have earlier seen simple data strcuture `pair` encoded in lambda calculus. 
* Recursive data structures can also be encoded.
  + List, Trees, etc.
* Mogensen–Scott encoding
  + Take constructors as arguments

In [ ]:
let nil = p "\\c.\\n.n"
let cons = p "\\h.\\t.\\c.\\n.c h t"

In [ ]:
let l2 = app [cons; two; app [cons; one; nil]] (* [2;1] *)

## Lists: Head and Tail

Empty list (`nil`) is `λc.λn.n` and non-empty list is `λc.λn.c h t`.

In [ ]:
let hd = lam "l" (app [var "l"; tru; nil])
let tl = lam "l" (app [var "l"; fls; nil])

In [ ]:
eval ~log:false @@ app [hd; l2];;
eval ~log:false @@ app [tl; l2];;
eval ~log:false @@ app [hd; nil];;
eval ~log:false @@ app [tl; nil]

## Lists: is_nil?

Empty list (`nil`) is `λc.λn.n` and non-empty list is `λc.λn.c h t`.

```ocaml
is_nil = λl.l (λx.λy.fls) tru
```

Similar idea to `is_zero` function

```ocaml
is_zero = λn.n (λy.fls) tru
```

## Lists: is_nil?

In [ ]:
let is_nil = lam "l" (app [var "l"; lam "x" (lam "y" fls); tru])

In [ ]:
eval ~log:false @@ app [is_nil; l2];;
eval ~log:false @@ app [is_nil; nil]

## Lists: Length

Empty list (`nil`) is `λc.λn.n` and non-empty list is `λc.λn.c h t`.

```ocaml
length = λf.λl.if is_nil l then zero else succ (f (tl l))
```

where `f` is the recursive function.

In [ ]:
let len = 
  let cond = app[is_nil; var "l"] in
  let flsc = app[scc; app[var "f"; app [tl; var "l"]]] in
  app [y; lam "f" (lam "l" (app [cond; zero; flsc]))]

In [ ]:
eval ~log:false @@ app [len; nil];;
eval ~log:false @@ app [len; l2];;

## Discussion

* Lambda calculus is Turing-complete
  + Most powerful language possible
  + Can represent pretty much anything in a "real" language
    * Using clever encodings
* But programs would be
  + Pretty slow (10000 + 1 → thousands of function calls)
  + Pretty inconvenient (difficult to recognize 10000 vs. 9999)
* In practice,
  + We use richer, more expressive languages
  + That include built-in primitives

<center>

<h1 style="text-align:center"> Fin. </h1>
</center>